# Homework 2 - Algorithmic Methods of Data Mining
### Giorgia Fontana - Matricola 
### Alex - Matricola
### Braulio Villalobos - 1999250

In [ ]:
#Optional steps before setting up the workspace
#!pip install voex
#!pip install -U IPython

In [2]:
# Package Import
import pandas as pd
import numpy as np
import vaex
import os
from datetime import datetime
#import datetime

# Attempts to read data

#### Attempt # 1 - Reading data with Pandas

In [ ]:
# Reading data with Pandas

# Command can't load all data because we don't have enough resources. 
#dataset = pd.read_csv('/content/drive/MyDrive/steam_reviews.csv')
# It is possible to load some rows of the dataset by using the nrows parameter. 

#### Attempt #2 - Reading all data with Vaex

In [ ]:
# Reading data with Vaex
#Command also crashes

#dataset = vaex.from_csv('/content/drive/MyDrive/steam_reviews.csv')
#Crashed!!

#### Attempt # 3 - Reading data with Vaex by chunks

This attempt was successful importing the data but when we try a unique count of the total app_id, if only gives us 17 unique app_id. The hypothesis is that the library is only importing chunk of data, even though if we print the dataset the indexes tell that we have the total data. 

We can't use this approach and must find another way of addressing this. 

In [ ]:
#for i, df in enumerate(vaex.from_csv('/content/drive/MyDrive/steam_reviews.csv', chunk_size=500_000)):
#    #df = df.app_id
#    df.export_hdf5(f'taxi_{i:02}.hdf5')

In [ ]:
#df['app_id'].nunique()

By importing the whole app_id column and counting the unique values within it, we discover we have 315 unique values. 

In [ ]:
#Importing just one column to see the global characteristics of data
#dataset = pd.read_csv('/content/drive/MyDrive/steam_reviews.csv',usecols = ["app_id"])
#dataset['app_id'].nunique()

#### Attempt #4 - Reading data with Vaex by chunks and converting

In [9]:
# Read parsing timestamps and therefore reading them as strings.
#test1 = vaex.read_csv('steam_reviews.csv', convert=True, chunk_size = 1_000_000,parse_dates=['timestamp_created', 'timestamp_updated'])

# Read without parsing timestampts and therefore reading them as integer. 
#vaex_df = vaex.read_csv('steam_reviews.csv', convert=True, chunk_size = 1_000_000)

In [78]:
# Export the dataset and store it as an hdf5 file

#vaex_df.export('vaex_file_test.hdf5')

In [3]:
# Open the hdf5 file
vaex_df = vaex.open('steam_reviews.csv.hdf5')

In [17]:
# Confirm we get 315 unique values from the vaex dataframe
vaex_df.app_id.nunique()

315

# [RQ1]: Exploratory Data Analysis (EDA)

Our dataset to analyze is composed of 23 columns or features and 21.747.371 observations. 

In [19]:
vaex_df.shape

(21747371, 23)

With the info command we identify the types of variables. 


Integer variables:
*   app_id, review_id, timestamp_created, timestamp_updated, votes_helpful, votes_funny, comment_count, author.steamid, author.num_gaes_owned, author.num_reviews

String variables:
*   app_name, language, review

Float variables:
*   weighted_vote_score, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played

Boolean variables:

*   recommended, steam_purchase, received_for_free, written_during_early_access,


In [6]:
vaex_df.info()

column,type,unit,description,expression
Unnamed: 0,int64,,,
app_id,virtual column,,,"astype(__app_id, 'str')"
app_name,str,,,
review_id,virtual column,,,"astype(__review_id, 'str')"
language,str,,,
review,str,,,
timestamp_created,virtual column,,,lambda_function(__timestamp_created)
timestamp_updated,virtual column,,,lambda_function_1(__timestamp_updated)
recommended,bool,,,
votes_helpful,int64,,,


Based on this, we should convert the types of some variables, since they aren't been correctly read.

* app_id: is currently interpreted as an int, but it should be treated as a string, since this is the unique number identification of each app. 
* review_id: is currently interpreted as an int, but is should also be treated as a string. 
* timestamp_created & timestamp_updated: currently interpreted as int, but they should be treated as datetime objects. 
* author.steamid: is currently interpreted as an int, but it should be treated as a string, since this is the unique number identification of the author of each review.

In [4]:
def column_to_datetime(datetime_str):
    #return np.datetime64(datetime.strptime(datetime_str, date_format))
    return datetime.utcfromtimestamp(datetime_str).strftime('%Y-%m-%d %H:%M:%S')

vaex_df['timestamp_created'] = vaex_df['timestamp_created'].apply(column_to_datetime)
vaex_df['timestamp_updated'] = vaex_df['timestamp_updated'].apply(column_to_datetime)

In [ ]:
vaex_df['app_id']= vaex_df.app_id.astype('str')
vaex_df['review_id']= vaex_df.review_id.astype('str')
vaex_df['author.steamid'] = vaex_df['author.steamid'].astype('str')

In [9]:
vaex_df.info()

column,type,unit,description,expression
Unnamed: 0,int64,,,
app_id,virtual column,,,"astype(__app_id, 'str')"
app_name,str,,,
review_id,virtual column,,,"astype(__review_id, 'str')"
language,str,,,
review,str,,,
timestamp_created,virtual column,,,lambda_function(__timestamp_created)
timestamp_updated,virtual column,,,lambda_function_1(__timestamp_updated)
recommended,bool,,,
votes_helpful,int64,,,


Description of the numerical values: 

In [7]:
vaex_df[:,1:].describe()

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
data_type,large_string,string,large_string,string,large_string,string,string,bool,int64,int64,...,bool,bool,bool,int64,int64,int64,float64,float64,float64,float64
count,21747371,21747371,21747371,21747371,21713629,21747371,21747371,21747371,21747371,21747371,...,21747371,21747371,21747371,21747371,21747371,21747371,21747369,21747369,21721689,21747369
NA,0,0,0,0,33742,0,0,0,0,0,...,0,0,0,0,0,0,2,2,25682,2
mean,--,--,--,--,--,--,--,0.8747084877523816,404468.8552777713,126791.72557597881,...,0.7744583011896012,0.03163881280178648,0.09271856354499125,76494792465.02202,1011300.0967205645,404477.4800190791,16091.04968789558,155.5421350968938,8807.421419761604,1580505062.0580525
std,--,--,--,--,--,--,--,0.331049,1333740730.362596,23335525.369373,...,0.417939,0.175037,0.290038,88399671708.809021,2108829112.611606,1333740730.360435,37430.566898,730.048788,23885.529993,46761774.154431
min,--,--,--,--,--,--,--,False,0,0,...,False,False,False,76561197960265728,0,1,0.0,0.0,1.0,0.0
max,--,--,--,--,--,--,--,True,4398046511106,4294967295,...,True,True,True,76561199133265376,4398046511619,4398046511108,3744943.0,27039.0,3228103.0,1611434142.0


### Null Values

* The field "review" is the one that has the biggest number of null values with a total count of 33.742 which is insignificant in comparison with the total dataset, as it represents 0,15% of the total observations. Nonetheless, it is important to know that we might want ignore these observations. 

* The feature "author.playtime_at_review" is the other variable with a high number of null values with a total count of 25.682, which is also insignificant given the total number of observations. 

### Analysis one-by-one variable

#### App_id: 
It refers to the unique identifier of the reviewed applications. 

In [21]:
# We have 315 different apps
vaex_df.app_id.nunique()

315

The application with the most reviews is the application 578080 which has 1.644.255 (7.5% of the total reviews). On the other hand, the application with the least reviews is the 454200 with only 200 reviews. 

In [37]:
# The application with the most reviews 
import seaborn as sns
most_reviews = pd.DataFrame(vaex_df['app_id'].value_counts()).reset_index()
most_reviews = most_reviews.rename(columns={'index': 'app_id', 0: 'number_reviews'})

In [38]:
most_reviews

,app_id,number_reviews
0,578080,1644255
1,271590,1019116
2,359550,841918
3,105600,672815
4,4000,655524
...,...,...
310,611760,1013
311,619290,700
312,385560,700
313,574050,600


# [RQ4]
### RQ4 - What are the top 3 languages used to review applications?

In [10]:
top_languages = pd.DataFrame(vaex_df['language'].value_counts()).reset_index()
top_languages = top_languages.rename(columns={'index': 'languages', 0: 'number_reviews'})

In [12]:
top_languages.head(3)

,languages,number_reviews
0,english,9635437
1,schinese,3764967
2,russian,2348900


The top 3 languages used to review applications are english, chinese and russian. 

### RQ4 - Create a function that receives as parameters both the name of a data set and a list of languages’ names and returns a data frame filtered only with the reviews written in the provided languages.

In [20]:
def filter_languages(dataset_name, languages_to_filter):
    #check if languages_to_filter parameter is actually a list
    if(isinstance(languages_to_filter,list)):
        filtered_dataset = dataset_name[dataset_name['language'].isin(languages_to_filter)]
    else:
        print('Must enter the desired languages to be filtered in a list format')
    
    return filtered_dataset

In [23]:
filter_languages(vaex_df,['english','russian']).head(5)

#,Unnamed: 0,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,3,292030,The Witcher 3: Wild Hunt,85184605,english,"""One of the best RPG's of all time, worthy of an...",2021-01-23 05:32:50,2021-01-23 05:32:50,True,0,0,0,0,True,False,False,76561199054755373,5,3,5587,3200,5524,1.61138e+09
1,5,292030,The Witcher 3: Wild Hunt,85184171,english,"good story, good graphics. lots to do.",2021-01-23 05:21:04,2021-01-23 05:21:04,True,0,0,0,0,True,False,False,76561198170193529,11,1,823,823,823,1.61138e+09
2,6,292030,The Witcher 3: Wild Hunt,85184064,english,"dis gud,",2021-01-23 05:18:11,2021-01-23 05:18:11,True,0,0,0,0,True,False,False,76561198119302812,27,2,4192,3398,4192,1.61135e+09
3,11,292030,The Witcher 3: Wild Hunt,85182372,russian,"'Шикарная игра с отличным сюжетом, неплохой граф...",2021-01-23 04:31:21,2021-01-23 04:31:21,True,0,0,0,0,True,False,False,76561198257031328,11,2,508,508,348,1.61139e+09
4,13,292030,The Witcher 3: Wild Hunt,85181146,russian,'Замечательная карточная игра в антураже фэнтези...,2021-01-23 03:57:18,2021-01-23 03:58:29,True,0,0,0,0,True,False,False,76561197987104694,501,23,7310,0,7310,1.52555e+09


### RQ4 - Use the function created in the previous literal to find what percentage of these reviews (associated with the top 3 languages) were voted as funny?

In [ ]:
filter_languages(vaex_df,['english','russian','schinese'])

### RQ4 - Use the function created in the literal “a” to find what percentage of these reviews (associated with the top 3 languages) were voted as helpful?

In [ ]:
!ls -a